# Set up the environment

In [1]:
import numpy as np
import scipy.stats
# import sklearn.manifold
import matplotlib.pyplot as plt
# from matplotlib.patches import Rectangle
import seaborn
# import pandas as pd
# import sklearn
import os
import pickle
import json
# from scipy.spatial import distance
from scipy.stats import spearmanr
# from sklearn.metrics.pairwise import manhattan_distances
# from sklearn.manifold import TSNE, MDS
# from sklearn.decomposition import PCA
# from mpl_toolkits.mplot3d import Axes3D



# Define a function to create a filename

In [2]:
def getFileName(name, n_samples, model_name, layer_name):
    return name \
        + "_{}_".format(n_samples) \
        + "_{}_".format(model_name) \
        + "_{}".format(layer_name)  \
        + ".npy"   

# Define a function to create the upper triangular of Input RDMs

In [3]:
def get_upper_triangular(rdm):
    num_conditions = rdm.shape[0] #num samples
    return rdm[np.triu_indices(num_conditions,1)] #take all above the main diagonal (excluding it), returns flattened version

# Load the model(s) and layers

In [4]:
#load the np file containing the shape of the activations
ROOT_PATH = '/mnt/raid/ni/agnessa/RSA/'
NR_OF_SAMPLES = 10000
json_file_layers=os.path.join(ROOT_PATH,'resnets_selected_layers.json')
with open(json_file_layers, "r") as fd:
    selected_layers = json.load(fd)

#get the name of the model and of the layers
num_layers = 16
model_begin = 16 #index of the first layer of the desired model
model_name = selected_layers[model_begin].get('model')
layer_names = []
for i in range(num_layers):
    layer_names.append(selected_layers[model_begin+i].get('layer'))

# Create model RDMs by correlating between Input RDMs from different layers and models

In [ ]:
RSA_matrix = np.ones((np.array(layer_names).shape[0],np.array(layer_names).shape[0])) #num layers x num layers
RSA_matrix[:] = np.nan

#1. get upper triangulars
#2. calculate the correlation distance (1-Spearman's coefficient) between the upper triangulars
#3. repeat for all pairs of layers (and models)

#for multiple models: replace by model & layer_1 and layer_2 loops
for layer_i in layer_names:
    for layer_j in layer_names:
    
        ## load RDMs ##
        RDM_PATH_i = os.path.join(ROOT_PATH, 'Input_RDM/' + getFileName('Input_RDM', NR_OF_SAMPLES, model_name, layer_i))
        RDM_PATH_j = os.path.join(ROOT_PATH, 'Input_RDM/' + getFileName('Input_RDM', NR_OF_SAMPLES, model_name, layer_j))        
#         np_load_old = np.load # save np.load
#         np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k) # modify the default parameters of np.load: the function is acting weird otherwise
        Input_RDM_i = np.load(RDM_PATH_i)
        Input_RDM_j = np.load(RDM_PATH_j)
#         np.load = np_load_old #revert back to old function
    
        ## get upper triangulars, without the 0 diagonal
        print('Getting the upper triangular of ->', layer_i)
        ut_rdm_i = get_upper_triangular(Input_RDM_i)
        print('Getting the upper triangular of ->', layer_j)
        ut_rdm_j = get_upper_triangular(Input_RDM_j)
            
        # Spearman correlation
        print('Calculating the correlation distance between ->', layer_i, 'and', layer_j)
        RSA_i_j = 1- spearmanr(ut_rdm_i,ut_rdm_j)[0]
        print('Finished the correlation distance between ->', layer_i, 'and', layer_j)
        
        # Save into a matrix 
        print('Saving the correlation distance between ->', layer_i, 'and', layer_j)
        RSA_matrix[np.where(np.array(layer_names)==layer_i)[0][0],np.where(np.array(layer_names)==layer_j)[0][0]] = RSA_i_j
        
#save model RDM
path = os.path.join(ROOT_PATH + 'Model_RDM/', getFileName('Model_RDM', NR_OF_SAMPLES, model_name, 'all'))
np.save(path,RSA_matrix)

        

Getting the upper triangular of -> layer1.0
Getting the upper triangular of -> layer1.0
Calculating the correlation distance between -> layer1.0 and layer1.0
Finished the correlation distance between -> layer1.0 and layer1.0
Saving the correlation distance between -> layer1.0 and layer1.0
Getting the upper triangular of -> layer1.0
Getting the upper triangular of -> layer1.1
Calculating the correlation distance between -> layer1.0 and layer1.1
Finished the correlation distance between -> layer1.0 and layer1.1
Saving the correlation distance between -> layer1.0 and layer1.1
Getting the upper triangular of -> layer1.0
Getting the upper triangular of -> layer1.2
Calculating the correlation distance between -> layer1.0 and layer1.2
Finished the correlation distance between -> layer1.0 and layer1.2
Saving the correlation distance between -> layer1.0 and layer1.2
Getting the upper triangular of -> layer1.0
Getting the upper triangular of -> layer2.0
Calculating the correlation distance betwe